In [2]:
using PlotlyJS
# using XLSX
include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")
# XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))

WebIO._IJuliaInit()

calculate_reserve_gcd_KPI (generic function with 1 method)

In [3]:
folder_path = joinpath("..","output", "solutions_v10.6u")
aux = "all"
# aux = join(string.("n_",[7, 228, 259, 289]),"_")

gcdi_KPI_adequacy = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcdi_KPI_adequacy.parquet"))
gcd_KPI_adequacy = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcd_KPI_adequacy.parquet"))
KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_KPI_reserve.parquet"))
gcdi_KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcdi_KPI_reserve.parquet"))
gcd_KPI_reserve = read_parquet_and_convert(joinpath(folder_path,"$(aux)_gcd_KPI_reserve.parquet"))


;

../output/solutions_v10.6u/all_gcdi_KPI_adequacy.parquet
../output/solutions_v10.6u/all_gcd_KPI_adequacy.parquet
../output/solutions_v10.6u/all_KPI_reserve.parquet
../output/solutions_v10.6u/all_gcdi_KPI_reserve.parquet
../output/solutions_v10.6u/all_gcd_KPI_reserve.parquet


In [4]:
gcdi_KPI_adequacy

Row,configuration,day,iteration,LLD_h,ENS_MWh,demand_MWh,CURD_h,CUR_MWh,RES_production_MWh,LGEN_MWh,objective_value,objective_value_uc,Δobjective_value,Δobjective_value_relative,objective_value_ref_conf,Δobjective_value_ref_conf,Δobjective_value_relative_ref_conf,mu
,Symbol,Int64,Symbol,Int64?,Float64?,Float64?,Int64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,base_ramp_storage_envelopes_up_0_dn_0,228,demand_86,0,0.0,80633.1,0,1.68754e-13,18146.1,176.233,1.70768e6,1.75438e6,-46702.8,-0.0266207,1.70768e6,0.0,0.0,0.0
2,base_ramp_storage_envelopes_up_0_dn_0,228,demand_91,1,33.6218,79730.8,0,1.68754e-13,18146.1,476.993,2.03317e6,1.75438e6,278784.0,0.158907,2.03317e6,0.0,0.0,0.0
3,base_ramp_storage_envelopes_up_0_dn_0,228,demand_48,0,0.0,79698.9,0,1.68754e-13,18146.1,436.468,1.6869e6,1.75438e6,-67483.5,-0.0384657,1.6869e6,0.0,0.0,0.0
4,base_ramp_storage_envelopes_up_0_dn_0,228,demand_21,0,0.0,79753.1,0,1.68754e-13,18146.1,654.161,1.69489e6,1.75438e6,-59495.5,-0.0339125,1.69489e6,0.0,0.0,0.0
5,base_ramp_storage_envelopes_up_0_dn_0,228,demand_16,1,43.4455,79602.9,0,1.68754e-13,18146.1,246.298,2.10767e6,1.75438e6,3.53284e5,0.201372,2.10767e6,0.0,0.0,0.0
6,base_ramp_storage_envelopes_up_0_dn_0,228,demand_28,1,12.4327,79662.5,0,1.68754e-13,18146.1,254.951,1.79813e6,1.75438e6,43747.9,0.0249363,1.79813e6,0.0,0.0,0.0
7,base_ramp_storage_envelopes_up_0_dn_0,228,demand_54,1,56.747,80924.2,0,1.68754e-13,18146.1,325.935,2.29202e6,1.75438e6,5.37637e5,0.306454,2.29202e6,0.0,0.0,0.0
8,base_ramp_storage_envelopes_up_0_dn_0,228,demand_24,0,0.0,80179.2,0,1.68754e-13,18146.1,341.183,1.70051e6,1.75438e6,-53874.2,-0.0307083,1.70051e6,0.0,0.0,0.0
9,base_ramp_storage_envelopes_up_0_dn_0,228,demand_26,1,29.3219,79519.8,0,1.68754e-13,18146.1,261.366,1.97042e6,1.75438e6,2.16041e5,0.123144,1.97042e6,0.0,0.0,0.0


In [5]:
CSV.write("gcdi_KPI_adequacy.csv",gcdi_KPI_adequacy)
CSV.write("KPI_reserve.csv",KPI_reserve)

"KPI_reserve.csv"

In [6]:
reserve_thrs = 0.1
group_by = [:configuration, :day, :iteration]

ov = leftjoin(
    gcdi_KPI_adequacy[:,union(group_by,[:objective_value, :Δobjective_value, :ENS_MWh])],
    combine(groupby(KPI_reserve, group_by), :required_r_relative => (x-> all(abs.(x) .<= reserve_thrs))  => :required_reserve_within_margins),
    on = group_by
)

eov = combine(groupby(ov, [:configuration, :day, :required_reserve_within_margins]), [:objective_value, :Δobjective_value, :ENS_MWh] .=> mean .=> [:EOV, :EΔOV, :EENS])
sort!(transform!(eov, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu), :mu)

Row,configuration,day,required_reserve_within_margins,EOV,EΔOV,EENS,mu
,Symbol,Int64,Bool?,Float64,Float64,Float64,Float64?
1,base_ramp_storage_envelopes_up_0_dn_0,228,true,1.70252e6,-51862.5,0.185575,0.0
2,base_ramp_storage_envelopes_up_0_dn_0,228,false,2.28644e6,5.32062e5,58.3065,0.0
3,base_ramp_storage_envelopes_up_0_dn_0,259,false,1.72163e6,5.14768e5,57.1775,0.0
4,base_ramp_storage_envelopes_up_0_dn_0,259,true,1.15736e6,-49501.6,0.0,0.0
5,base_ramp_storage_envelopes_up_0_dn_0,27,false,2.14747e6,1.50562e6,153.1,0.0
6,base_ramp_storage_envelopes_up_0_dn_0,27,true,1.44906e6,8.07215e5,83.509,0.0
7,base_ramp_storage_envelopes_up_0_dn_0,289,false,2.3441e6,1.19677e6,124.704,0.0
8,base_ramp_storage_envelopes_up_0_dn_0,289,true,1.62888e6,4.81553e5,53.5944,0.0
9,base_ramp_storage_envelopes_up_0_dn_0,7,false,1.90741e6,1.20026e6,121.517,0.0


In [7]:
filter(row ->  row.day == 1 && row.iteration == :demand_10, ov)

Row,configuration,day,iteration,objective_value,Δobjective_value,ENS_MWh,required_reserve_within_margins
,Symbol,Int64,Symbol,Float64?,Float64?,Float64?,Bool?


In [8]:
plot(eov, x = :mu, y = :EΔOV, color = :day, facet_col = :required_reserve_within_margins)
# p2 = plot(eov, x = :mu, y = :EENS, color = :day, facet_col = :required_reserve_within_margins)


data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, yaxis, and yaxis2"

In [9]:
plot(eov, x = :mu, y = :EENS, color = :day, facet_col = :required_reserve_within_margins)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, yaxis, and yaxis2"

In [10]:
function redispatch_prob(r_relative_min, r_relative_max, required_reserve, number_key)
    out =  combine(groupby(required_reserve, [:configuration, :mu]), [:required_r_relative, :redispatch_needed] => ((x,y) -> 100*sum((x.>=r_relative_min).*(x.<=r_relative_max).*y)/sum((x.>=r_relative_min).*(x.<=r_relative_max))) => :redispatch_probability)
    out.r_max_relative .= number_key
    return out
end

redispatch_prob (generic function with 1 method)

In [11]:
redispatch_probability = vcat(
    vcat([redispatch_prob(0, r_max, KPI_reserve, r_max) for r_max in 0.0:0.01:0.15]...),
    vcat([redispatch_prob(-r_max, 0, KPI_reserve, -r_max) for r_max in 0.0:0.01:0.15]...)
)
sort!(redispatch_probability, :r_max_relative)
# redispatch_probability = vcat([redispatch_prob(-r_max, r_max, required_reserve, r_max) for r_max in 0.0:0.01:0.2]...)
p1 = plot(redispatch_probability, x = :r_max_relative, y = :redispatch_probability, color = :mu, Layout(yaxis_title = "Probability of redispatch) [%]"),mode="markers+lines")

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

In [12]:
p2 = plot(filter(:day => x -> x in [7, 1], KPI_reserve), x = :required_r_relative, y = :mu, facet_col = :redispatch_needed, facet_row = :day, mode = "markers", Layout(height=800))

data: [
  "scatter with fields mode, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, legend, margin, template, xaxis, xaxis2, yaxis, and yaxis2"

In [13]:
p1 = plot(gcd_KPI_adequacy, x = :mu, y = :EOV, color = :day)
p2 = plot(gcd_KPI_adequacy, x = :mu, y = :EENS, color = :day)
p3 = plot(gcd_KPI_adequacy, x = :mu, y = :ECUR, color = :day)
p4 = plot(gcd_KPI_adequacy, x = :mu, y = :ELGEN, color = :day)
# p4 = plot(gcd_KPI_adequacy, x = :mu, y = :ELGEN, color = :day)
# p3 = plot(KPI, x = :mu, y = :LOLE, color = :day)
[p1 p2 p4]
# savefig([p1 p2 p3 p4], "adecuacy KPIs v1.png", width=12*scale, height=6*scale)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, xaxis3, yaxis1, yaxis2, and yaxis3"

In [14]:
p1 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_up_ratio, color = :day)
p2 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_dn_ratio, color = :day)
p3 = plot(gcd_KPI_reserve, x = :mu, y = :E_delivered_r_up_MWh, color = :day)
p4 = plot(gcd_KPI_reserve, x = :mu, y = :E_deliverered_dn_MWh, color = :day)
# [p1 p2; p3 p4]
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [15]:
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :mu, color = :iteration)
# p2 = plot(gcdi_KPI_adequacy, y = :ENS_MWh, x = :mu, color = :iteration)
# p3 = plot(gcdi_KPI_adequacy, y = :LGEN_MWh, x = :mu, color = :iteration)
# p4 = plot(gcdi_KPI_adequacy, y = :CUR_MWh, x = :mu, color = :iteration)
# [p1 p2; p3 p4]

In [16]:
p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :mu, facet_col = :day, color = :iteration)
[p1;]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and y

In [17]:
# function check_loq_value(df, field, value, accuracy)
#     filter = (df[!,field] .> value) .& (df[!,field] .< Inf) .& (.!isnan.(df[!,field]))
#     filter = (df[!,field].*filter .-value) .< accuracy
#     println(all(filter))
#     return df[.!filter,:]

# end
# # Check
# accuracy = 0.000001
# check_loq_value(KPI_reserve, :delivered_r_up_ratio, 1, accuracy)
# check_loq_value(KPI_reserve, :delivered_r_dn_ratio, 1, accuracy)
;
# balance = (KPI_reserve.required_r_up_MW .-KPI_reserve.required_r_dn_MW) .- (KPI_reserve.delivered_r_up_MW .- KPI_reserve.delivered_r_dn_MW .+ KPI_reserve.LOL_MW.*(required.>=0))
# println(all(abs.(balance) .<= accuracy))
# Todo: re-calcualte balance balance
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration) 
# p1 = plot(gcdi_KPI_adequacy, y = :Δobjective_valuef, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)
# p1 = plot(gcdi_KPI_adequacy, y = :objective_value, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# p2 = plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# p3= plot(gcdi_KPI_adequacy, y = :Δobjective_value_relative, x = :ENS_MWh, mode = "markers", color= :configuration, text =:iteration)
# relayout([p1;p2;p3], height=800)

### Checking reserve delivey

In [18]:
day_ = 7
iteration_ = :demand_40
config_ = :base_ramp_storage_envelopes_up_0_3_dn_0_3
KPI_reserve_ = KPI_reserve[(KPI_reserve.configuration .== config_) .& (KPI_reserve.day .== day_) .& (KPI_reserve.iteration .== iteration_), :]
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_up_MW, name = "required_r_up_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_up_MW, name = "delivered_r_up_MW")]
p1 = plot(p, Layout(yaxis_title="reserve up MW", xaxis_title="hour"))
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_dn_MW, name = "required_r_dn_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_dn_MW, name = "delivered_r_dn_MW")]
p2 = plot(p, Layout(yaxis_title="reserve dn MW", xaxis_title="hour"))
[p1 p2]

data: [
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [19]:
group_by_big= [:configuration, :day]
reserve_ = combine(groupby(s_uc.reserve, union(group_by_big, [:resource])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_MW,:reserve_down_MW])
combine(groupby(reserve_, union(group_by_big, [])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_total_MW,:reserve_down_total_MW])
leftjoin!(reserve_,
combine(groupby(reserve_, union(group_by_big, [])), [:reserve_up_MW,:reserve_down_MW]  .=> sum .=> [:reserve_up_total_MW,:reserve_down_total_MW]),
on = group_by_big
)

reserve_.reserve_down_relative = reserve_.reserve_down_MW ./ reserve_.reserve_down_total_MW
reserve_.reserve_up_relative = reserve_.reserve_up_MW ./ reserve_.reserve_up_total_MW
transform!(reserve_, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
sort!(reserve_, :mu)
;

LoadError: UndefVarError: `s_uc` not defined

In [ ]:
fieldy = :resource
fieldx = :reserve_up_relative
df = reserve_[reserve_.day .== 7,:] 
plot([scatter(
          x= df[df[!,fieldy] .== r, :mu], y=df[df[!,fieldy] .== r, fieldx],
          stackgroup=map_stack_group(r,"required"), mode="lines", name = r,
          line=attr(width=1, color=color_discrete_map(r), shape = "line")
      ) for r in unique(df[!,fieldy])],
    Layout(yaxis_title=string(fieldx), xaxis_title=string(:mu), title = string(fieldx))
)
